In [1]:
# !pip install -U git+https://github.com/huggingface/transformers.git
# !pip install -U git+https://github.com/huggingface/accelerate.git
# !pip install -U bitsandbytes 
# #accelerate transformers
# !pip install llm2vec


In [2]:
# !pip install llm2vec
# !pip install bitsandbytes==0.43.2

In [1]:
!pip install llm2vec
# !pip install transformers==4.43.1 bitsandbytes==0.43.0 accelerate==0.21.0
!pip install transformers==4.44.2 bitsandbytes==0.43.0 accelerate==0.30.0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 96.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 85.1 MB/s eta 0:00:00:00:01
  Created wheel for llm2vec: filename=llm2vec-0.2.3-py3-none-any.whl size=31034 sha256=502481a8ce3dd65ec5734dc261bb763c827789145dd12cd49cb784f457178345
  Stored in directory: /root/.cache/pip/wheels/a7/02/f3/9b2482fb71501a9dce285b026db2cd76c7b542b047d891dcb8
Successfully built llm2vec
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.0
    Uninstalling transformers-4.47.0:
      Successfully uninstalled transformers-4

In [2]:
import transformers, accelerate, bitsandbytes
print(transformers.__version__)  
print(accelerate.__version__)    
print(bitsandbytes.__version__)  

4.44.2
0.30.0
0.43.0


In [3]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hugginface_key")
secret_value_1 = user_secrets.get_secret("wandb_key")

In [4]:
from huggingface_hub import login
login(token=secret_value_0)

In [5]:
import pandas as pd

df = pd.read_parquet("/kaggle/input/unlp-2025-shared-task-span-identification/train.parquet")#'train.parquet')
cv = pd.read_csv("/kaggle/input/span-ident-cv-split-csv/cv_split.csv")#'cv_split.csv')
df = df.merge(cv, on='id', how='left')

df_test = pd.read_csv("/kaggle/input/unlp-2025-shared-task-span-identification/test.csv")#'test.csv')

In [6]:
import spacy

from spacy.training.iob_utils import biluo_to_iob, doc_to_biluo_tags
from tqdm.autonotebook import tqdm
tqdm.pandas()

df.trigger_words = df.trigger_words.apply(lambda x: [] if x is None else x)

In [8]:
PRETRAINED_MODEL = 'McGill-NLP/LLM2Vec-Meta-Llama-31-8B-Instruct-mntp'
# PRETRAINED_MODEL = "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp"
MAX_LEN = 512

In [9]:
from transformers import AutoTokenizer
import pandas as pd
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [10]:
from transformers import AutoTokenizer
import pandas as pd

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

def convert_to_seq_labeling(text, tokenizer, trigger_spans=None):
    tokenized_output = tokenizer(
        text, return_offsets_mapping=True, add_special_tokens=True, max_length=MAX_LEN,
        truncation=True, padding=False
    )
    tokens = tokenized_output["input_ids"]
    offsets = tokenized_output["offset_mapping"]

    # Get subword tokenized versions of the text
    token_strings = tokenizer.convert_ids_to_tokens(tokens)

    
    # Initialize labels as 'O'
    labels = [0] * len(tokens)

    if trigger_spans is not None:
        # Assign 'TRIGGER' to overlapping tokens
        for start, end in trigger_spans:
            for i, (tok_start, tok_end) in enumerate(offsets):
                if tok_start == 0 and tok_end == 0:
                    continue
                if tok_start < end and tok_end > start:  # If token overlaps with the trigger span
                    labels[i] = 1

    tokenized_output['labels'] = labels
    return tokenized_output

In [11]:
from tqdm.autonotebook import tqdm

tqdm.pandas()

df['seq_labels'] = df.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, row['trigger_words']), axis=1)
for column in df.seq_labels.iloc[0].keys():
    df[column] = df.seq_labels.apply(lambda x: x.get(column))

df_test['seq_labels'] = df_test.progress_apply(lambda row: convert_to_seq_labeling(row['content'], tokenizer, None), axis=1)
for column in df_test.seq_labels.iloc[0].keys():
    df_test[column] = df_test.seq_labels.apply(lambda x: x.get(column))

  0%|          | 0/3822 [00:00<?, ?it/s]

  0%|          | 0/5735 [00:00<?, ?it/s]

In [12]:
from datasets import Dataset
import numpy as np

df['is_valid'] = df.fold == 4

columns = list(df.seq_labels.iloc[0].keys()) + ['content', 'trigger_words']
ds_train = Dataset.from_pandas(df[df.is_valid==0][columns].reset_index(drop=True))
ds_valid = Dataset.from_pandas(df[df.is_valid==1][columns].reset_index(drop=True))

columns = list(df.seq_labels.iloc[0].keys()) + ['content']
ds_test = Dataset.from_pandas(df_test[columns].reset_index(drop=True))

# Model

In [14]:
# from kaggle_secrets import UserSecretsClient
# user_secrets = UserSecretsClient()
# secret_value_0 = user_secrets.get_secret("hugginface_key")
# secret_value_1 = user_secrets.get_secret("wandb_key")

# from huggingface_hub import login
# login(token=secret_value_0)

In [15]:
# !pip install llm2vec
# # !pip install transformers==4.43.1 bitsandbytes==0.43.0 accelerate==0.21.0
# !pip install transformers==4.44.2 bitsandbytes==0.43.0 accelerate==0.30.0

In [13]:
import transformers, accelerate, bitsandbytes
print(transformers.__version__)  # Should be ≥4.37.0
print(accelerate.__version__)    # Should be ≥0.27.0
print(bitsandbytes.__version__)  # Should be ≥0.43.0

4.44.2
0.30.0
0.43.0


In [14]:
from transformers import Trainer

In [15]:
from transformers import BitsAndBytesConfig
import torch

In [16]:
from transformers import AutoTokenizer
from llm2vec import LLM2Vec

# original model in repo
# MODEL_NAME_OR_PATH = "meta-llama/Llama-2-7b-chat-hf"
# PEFT_ADDR = "McGill-NLP/LLM2Vec-Llama-2-7b-chat-hf-mntp"

MODEL_NAME_OR_PATH = "meta-llama/Llama-3.1-8B-Instruct"
PEFT_ADDR = "McGill-NLP/LLM2Vec-Meta-Llama-31-8B-Instruct-mntp"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME_OR_PATH)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

nf4_config = BitsAndBytesConfig(
   load_in_4bit=True,
   bnb_4bit_quant_type="nf4",
   bnb_4bit_use_double_quant=False,
   bnb_4bit_compute_dtype=torch.float16
)

l2v = LLM2Vec.from_pretrained(
    base_model_name_or_path=MODEL_NAME_OR_PATH,
    peft_model_name_or_path=PEFT_ADDR,
    merge_peft=False,
    enable_bidirectional=True,
    # load_in_4bit=True,
    quantization_config=nf4_config,
    device_map="auto",

)

tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/837 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/168M [00:00<?, ?B/s]

In [17]:
import torch
from transformers import PreTrainedModel
from transformers.modeling_outputs import TokenClassifierOutput

class ModelForWordTask(PreTrainedModel):
    def __init__(self, config, base_model, merge_subwords=False, torch_dtype=torch.float32):
        super().__init__(config)
        self.model = base_model  # This is the bidirectional model from LLM2Vec (i.e. l2v.model)
        self.merge_subwords = merge_subwords

        # Use classifier_dropout from config if provided.
        classifier_dropout = config.classifier_dropout if hasattr(config, "classifier_dropout") else 0.1
        self.dropout = torch.nn.Dropout(classifier_dropout)
        self.num_labels = config.num_labels
        # Linear classification head for token-level predictions.
        self.classifier = torch.nn.Linear(config.hidden_size, config.num_labels).to(torch_dtype)
        self.post_init()

    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        labels=None,
        **kwargs,
    ):
      
        outputs = self.model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            # token_type_ids=token_type_ids,
            **kwargs,
        )
        # Get token-level representations.
        hidden_states = outputs[0]  # shape: [batch_size, seq_len, hidden_dim]
        hidden_states = self.dropout(hidden_states)
        logits = self.classifier(hidden_states)

        loss = None
        if labels is not None:
            labels = labels.to(logits.device)
            loss_fct = torch.nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
        return TokenClassifierOutput(loss=loss, logits=logits, hidden_states=hidden_states)



In [19]:
from transformers import AutoConfig

tc_config = AutoConfig.from_pretrained(MODEL_NAME_OR_PATH)
tc_config.num_labels = 2
tc_config.id2label = {0: 0, 1: 1}
tc_config.label2id = {0: 0, 1: 1}
tc_config.classifier_dropout = 0.1


model = ModelForWordTask(
    config=tc_config,
    base_model=l2v.model,
    merge_subwords=True,  # as per config
    torch_dtype=torch.float16,
)

In [20]:
from peft import get_peft_config, prepare_model_for_kbit_training, PeftModel, PeftConfig, get_peft_model, LoraConfig, TaskType

lora_config = LoraConfig(
    r=16,                    # low-rank dimension
    lora_alpha=16,           # scaling factor for LoRA activations
    lora_dropout=0.05,
    bias="none",
    inference_mode=False,
    task_type=TaskType.TOKEN_CLS,
    target_modules=['o_proj', 'v_proj', 'q_proj', 'k_proj']#, 'gate_proj', 'down_proj', 'up_proj']
)

# Prepare the model for k-bit (quantized) training.
model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 41,951,234 || all params: 7,546,884,100 || trainable%: 0.5559


In [21]:
model

PeftModelForTokenClassification(
  (base_model): LoraModel(
    (model): ModelForWordTask(
      (model): PeftModel(
        (base_model): LoraModel(
          (model): LlamaBiModel(
            (embed_tokens): Embedding(128256, 4096)
            (layers): ModuleList(
              (0-31): 32 x ModifiedLlamaDecoderLayer(
                (self_attn): ModifiedLlamaSdpaAttention(
                  (q_proj): lora.Linear4bit(
                    (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.05, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=4096, out_features=16, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=16, out_features=4096, bias=False)
                    )
                    (lora_embedding_A): Param

In [22]:
# print(next(model.parameters()))

# Training

In [23]:
import os
import random
import numpy as np
import torch

def set_seeds(seed):
    """Set seeds for reproducibility """
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        

set_seeds(seed=42)

In [24]:
import math
from transformers import TrainingArguments
from typing import Any
from transformers.trainer_utils import EvalPrediction


train_args = TrainingArguments(
    output_dir='model_checkpoints_llama3_8b_binary',
    logging_dir='./model_logs_llama3_8b_binary',
    learning_rate=2e-5,
    weight_decay=0.01,
    lr_scheduler_type='cosine',
    warmup_ratio=0.0,
    num_train_epochs=3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=8,
    # bf16=True,
    fp16=True,
    report_to="wandb",
    optim='adamw_8bit',
    eval_strategy='steps',
    save_strategy="steps",
    eval_steps=200,
    logging_steps=20,
    save_steps=200,
    save_total_limit=10,
    metric_for_best_model='eval_f1',
    greater_is_better=True,
    load_best_model_at_end=True,
)

In [25]:
import wandb
wandb.login(key=secret_value_1)

# Initialize with team/entity
wandb.init(
    project="unlp-span-ident-task",
    entity="IASA-BA-Diploma-Ivan-Bashtovyi", 
    name='llm2vec-llama3-8b-binary-seq-trunc',
    settings=wandb.Settings(init_timeout=180)  # Increase timeout
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ivanbashtovyi1 (IASA-BA-Diploma-Ivan-Bashtovyi). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [26]:
from itertools import chain

positive_class_balance = pd.Series(list(chain(*df.labels.tolist()))).mean()
positive_class_balance

0.23897133895827039

In [27]:
import math
from transformers import Trainer, TrainingArguments
from typing import Any
from tqdm.autonotebook import tqdm
from transformers.trainer_utils import EvalPrediction

def extract_chars_from_spans(spans):
    """
    Given a list of spans (each a tuple (start, end)),
    return a set of character indices for all spans.
    """
    char_set = set()
    for start, end in spans:
        # Each span covers positions start, start+1, ..., end-1.
        char_set.update(range(start, end))
    return char_set

class SpanEvaluationTrainer(Trainer):
    def __init__(
        self,
        model: Any = None,
        args: TrainingArguments = None,
        data_collator: Any = None,
        train_dataset: Any = None,
        eval_dataset: Any = None,
        tokenizer: Any = None,
        desired_positive_ratio: float = 0.25,
        **kwargs,
    ):
        """
        Initialize the Trainer with our custom compute_metrics.
        """
        super().__init__(
            model=model,
            args=args,
            data_collator=data_collator,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=tokenizer,
            compute_metrics=self.compute_metrics,  # assign our custom compute_metrics
            **kwargs,
        )
        self.desired_positive_ratio = desired_positive_ratio

    def _calculate_inner_metric(self, gt_spans_all, pred_spans_all):
        total_true_chars = 0
        total_pred_chars = 0
        total_overlap_chars = 0
        for true_spans, pred_spans in zip(gt_spans_all, pred_spans_all):
            if isinstance(true_spans, str):
                try:
                    true_spans = eval(true_spans)
                except Exception:
                    true_spans = []
                    
            # Convert spans to sets of character indices.
            true_chars = extract_chars_from_spans(true_spans)
            pred_chars = extract_chars_from_spans(pred_spans)
            
            total_true_chars += len(true_chars)
            total_pred_chars += len(pred_chars)
            total_overlap_chars += len(true_chars.intersection(pred_chars))
            
            union_chars = true_chars.union(pred_chars)
            
        # Compute precision, recall, and F1.
        precision = total_overlap_chars / total_pred_chars if total_pred_chars > 0 else 0
        recall = total_overlap_chars / total_true_chars if total_true_chars > 0 else 0
        f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
        
        metrics = {
            "precision": precision,
            "recall": recall,
            "f1": f1
        }
        return metrics

    def _find_optimal_threshold(self, probabilities, labels):
        """Finds the threshold that achieves the desired positive class balance."""
        best_th = 0.5  # Default starting point
        best_diff = float("inf")
        optimal_th = best_th
        
        for thold in np.linspace(0.01, 0.99, num=100):
            predictions = (probabilities[:, :, 1] >= thold).astype(int)
            true_predictions = [
                [p for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
            total_pos = sum([sum(row for row in prediction) for prediction in true_predictions])
            total = sum([len(prediction) for prediction in true_predictions])
            
            positive_ratio = total_pos / total if total > 0 else 0
            
            diff = abs(positive_ratio - self.desired_positive_ratio)
            if diff < best_diff:
                best_diff = diff
                optimal_th = thold
        
        return optimal_th
        
        
    def compute_metrics(self, eval_pred: EvalPrediction) -> dict:
        eval_dataset = self.eval_dataset
        logits, labels = eval_pred
        probabilities = torch.softmax(torch.tensor(logits), dim=-1).cpu().numpy()
    
        #thresholds = np.linspace(0.1, 0.5, num=41)
        thresholds = [self._find_optimal_threshold(probabilities, labels)]
        results = []
        best_f1 = -1
        best_th = 0
        best_metrics = None
    
        for thold in tqdm(thresholds):
            # Apply thresholding instead of argmax
            predictions = (probabilities[:, :, 1] >= thold).astype(int)
    
            true_predictions = [
                [p for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
    
            pred_spans_all = []
            for pred, offsets in zip(true_predictions, eval_dataset['offset_mapping']):
                samplewise_spans = []
                current_span = None
                for token_label, span in zip(pred, offsets):
                    if token_label == 1:  # If the current token is labeled as an entity (1)
                        if current_span is None:
                            current_span = [span[0], span[1]]  # Start a new span
                        else:
                            current_span[1] = span[1]  # Extend the span to include the current token
                    else:  # If token_label == 0 (not an entity)
                        if current_span is not None:
                            samplewise_spans.append(tuple(current_span))  # Save completed span
                            current_span = None  # Reset for the next entity
    
                # If the last token was part of a span, save it
                if current_span is not None:
                    samplewise_spans.append(tuple(current_span))
    
                pred_spans_all.append(samplewise_spans)
    
            # Store results for this threshold
            current_metrics = self._calculate_inner_metric(eval_dataset['trigger_words'], pred_spans_all)
            if current_metrics['f1'] >= best_f1:
                best_f1 = current_metrics['f1']
                best_th = thold
                best_metrics = current_metrics
                best_metrics['thold'] = thold
                
            
            results.append(current_metrics)
        return best_metrics

In [28]:
from transformers import DataCollatorForTokenClassification, Trainer

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

trainer = SpanEvaluationTrainer(
    model=model,
    args=train_args,
    train_dataset=ds_train,
    eval_dataset=ds_valid,
    data_collator=data_collator,
    tokenizer=tokenizer,
    desired_positive_ratio=positive_class_balance
)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:479: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification, AutoConfig, BitsAndBytesConfig
from peft import PeftModel, LoraConfig, TaskType, prepare_model_for_kbit_training, get_peft_model

# Define label mappings (retained from your original code)
id2label = {0: 0, 1: 1}
label2id = {0: 0, 1: 1}

# Quantization configuration (NF4)
nf4_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load base model with token classification head (quantized)
config = AutoConfig.from_pretrained(PRETRAINED_MODEL, trust_remote_code=True)
model = AutoModelForTokenClassification.from_pretrained(
    PRETRAINED_MODEL,
    config=config,
    id2label=id2label,
    label2id=label2id,
    quantization_config=nf4_config,
    torch_dtype=torch.bfloat16,
    device_map="cuda" if torch.cuda.is_available() else "cpu",
)

# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA Configuration
lora_config = LoraConfig(
    r=32,  # Low-rank matrix dimension
    lora_alpha=16,  # Scaling factor
    lora_dropout=0.05,
    bias="none",
    inference_mode=False,
    task_type=TaskType.TOKEN_CLS,
    target_modules=["o_proj", "v_proj", "q_proj", "k_proj"]
)

# Apply LoRA fine-tuning
model = get_peft_model(model, lora_config)

# Print trainable parameters
model.print_trainable_parameters()

In [ ]:
trainer.model = model.cuda().eval()

In [ ]:
valid_preds = trainer.predict(ds_valid)

In [ ]:
valid_preds.predictions.shape, valid_preds.label_ids.shape

In [ ]:
trainer.compute_metrics((valid_preds.predictions, valid_preds.label_ids))

In [ ]:
test_preds = trainer.predict(ds_test)

In [ ]:
test_probabilities = torch.softmax(torch.tensor(test_preds.predictions), dim=-1).cpu().numpy()

In [ ]:
trainer._find_optimal_threshold(test_probabilities, test_preds.label_ids)

In [ ]:
final_th = (0.4554+0.4257575)/2 - 0.15
final_th

In [ ]:
def inference_aggregation(probabilities, labels, offset_mappings, thold):
    predictions = (probabilities[:, :, 1] >= thold).astype(int)
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)
    ]
    pred_spans_all = []
    for pred, offsets in zip(true_predictions, offset_mappings):
        samplewise_spans = []
        current_span = None
        for token_label, span in zip(pred, offsets):
            if token_label == 1:  # If the current token is labeled as an entity (1)
                if current_span is None:
                    current_span = [span[0], span[1]]  # Start a new span
                else:
                    current_span[1] = span[1]  # Extend the span to include the current token
            else:  # If token_label == 0 (not an entity)
                if current_span is not None:
                    samplewise_spans.append(tuple(current_span))  # Save completed span
                    current_span = None  # Reset for the next entity
        
                    # If the last token was part of a span, save it
        if current_span is not None:
            samplewise_spans.append(tuple(current_span))
        
        pred_spans_all.append(samplewise_spans)
    return [str(row) for row in pred_spans_all]

In [ ]:
valid_probabilities = torch.softmax(torch.tensor(valid_preds.predictions), dim=-1).cpu().numpy()
valid_results = inference_aggregation(valid_probabilities, valid_preds.label_ids, ds_valid['offset_mapping'], final_th)

In [ ]:
import pandas as pd
import pandas.api.types
from sklearn.metrics import f1_score
import ast


class ParticipantVisibleError(Exception):
    """Custom exception for participant-visible errors."""
    pass


def score(solution: pd.DataFrame, submission: pd.DataFrame, row_id_column_name: str) -> float:
    """
    Compute span-level F1 score based on overlap.

    Parameters:
    - solution (pd.DataFrame): Ground truth DataFrame with row ID and token labels.
    - submission (pd.DataFrame): Submission DataFrame with row ID and token labels.
    - row_id_column_name (str): Column name for the row identifier.

    Returns:
    - float: The token-level weighted F1 score.

    Example:
    >>> solution = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (725, 831)], [(300, 312)], []]
    ... })
    >>> submission = pd.DataFrame({
    ...     "id": [1, 2, 3],
    ...     "trigger_words": [[(612, 622), (700, 720)], [(300, 312)], [(100, 200)]]
    ... })
    >>> score(solution, submission, "id")
    0.16296296296296295
    """
    if not all(col in solution.columns for col in ["id", "trigger_words"]):
        raise ValueError("Solution DataFrame must contain 'id' and 'trigger_words' columns.")
    if not all(col in submission.columns for col in ["id", "trigger_words"]):
        raise ValueError("Submission DataFrame must contain 'id' and 'trigger_words' columns.")
    
    def safe_parse_spans(trigger_words):
        if isinstance(trigger_words, str):
            try:
                return ast.literal_eval(trigger_words)
            except (ValueError, SyntaxError):
                return []
        if isinstance(trigger_words, (list, tuple, np.ndarray)):
            return trigger_words
        return []

    def extract_tokens_from_spans(spans):
        tokens = set()
        for start, end in spans:
            tokens.update(range(start, end))
        return tokens
    
    solution = solution.copy()
    submission = submission.copy()

    solution["trigger_words"] = solution["trigger_words"].apply(safe_parse_spans)
    submission["trigger_words"] = submission["trigger_words"].apply(safe_parse_spans)

    merged = pd.merge(
        solution,
        submission,
        on="id",
        suffixes=("_solution", "_submission")
    )

    total_true_tokens = 0
    total_pred_tokens = 0
    overlapping_tokens = 0

    for _, row in merged.iterrows():
        true_spans = row["trigger_words_solution"]
        pred_spans = row["trigger_words_submission"]

        true_tokens = extract_tokens_from_spans(true_spans)
        pred_tokens = extract_tokens_from_spans(pred_spans)

        total_true_tokens += len(true_tokens)
        total_pred_tokens += len(pred_tokens)
        overlapping_tokens += len(true_tokens & pred_tokens)

    precision = overlapping_tokens / total_pred_tokens if total_pred_tokens > 0 else 0
    recall = overlapping_tokens / total_true_tokens if total_true_tokens > 0 else 0
    f1 = (2 * precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    return f1

In [ ]:
from copy import deepcopy

df_gt = df[df.fold==4][['id', 'trigger_words']].reset_index(drop=True)
df_pred = deepcopy(df_gt)
df_pred['trigger_words'] = valid_results
score(df_gt, df_pred, row_id_column_name='id')

In [ ]:
test_results = inference_aggregation(test_probabilities, test_preds.label_ids, ds_test['offset_mapping'], final_th)

In [ ]:
ss = pd.read_csv('sample_submission.csv')
ss['trigger_words'] = test_results

In [ ]:
ss.to_csv('submissions/gemma2-27b-binary-cv0.627blackmagic.csv', index=False)

In [ ]:
import pickle

pickle.dump(valid_preds, open('local_preds/valid_preds_gemma2_binary.pkl', 'wb'))
pickle.dump(test_preds, open('local_preds/test_preds_gemma2_binary.pkl', 'wb'))